


---
---

# Depthwise Convolution: Ortaya Çıkış Gerekçesi ve İşlevsel Değeri

---
----


## 1. Problemin Tanımı: Standart Konvolüsyonun Hesaplama Maliyeti

#### Klasik 2D konvolüsyon (“full convolution”) iki temel görevi aynı anda yerine getirir:


* Uzamsal bilgi çıkarımı (spatial filtering)

* Kanallar arası etkileşim (channel mixing)

Bu iki görevi tek bir operasyon içinde birleştirdiği için parametre sayısı ve hesaplama maliyeti çok yüksektir.

**Bir standart 3×3 konvolüsyonun parametre sayısı:**
```bash
Paramsstandard ​= C_in × C_out ​× 3×  3
```


* Bu, özellikle mobil cihazlar, gömülü sistemler ve gerçek zamanlı uygulamalarda kabul edilemez derecede yüksek hesaplama yükü oluşturur.
**Bu nedenle araştırmacılar, şu soruyu gündeme getirmiştir:**
>**“Uzamsal filtreleme ile kanal karıştırmayı aynı anda yapmak zorunda mıyız?”**

---

## 2. Çözüm Yaklaşımı: İşlevi Ayrıştırma Fikri

Araştırmalar, standart konvolüsyonun aslında iki farklı bileşenden oluştuğunu göstermiştir:

#### **1-) Spatial Filtering (3×3)**

* Her kanal üzerinde lokal özellik çıkarımı yapılır.

#### **2-) Channel Mixing (1×1)**

* Farklı kanallar arasında ilişki kurulup yeni özellik haritaları üretilir.

Bu iki bileşenin tek bir işlem içinde birleştirilmesi gereksiz maliyet yaratmaktadır.
**Dolayısıyla çözüm şu şekilde formüle edilmiştir:**

* Uzamsal işlemi ve kanal karışımını birbirinden ayırmak.

* Bu ayrıştırmanın ilk adımı Depthwise Convolution ile gerçekleşmiştir

---

## 3. Depthwise Convolution’un Temel Fikri

* Depthwise convolution, her giriş kanalının kendi 3×3 filtresiyle bağımsız olarak işlenmesi prensibine dayanır.

Yani:

* Kanal 1 → kendi filtresi

* Kanal 2 → kendi filtresi

- …

* Kanal C_in → kendi filtresi

**Bu durumda parametre sayısı:**
```bash
ParamsDW​=C_in ​× 3 × 3
```

**Bu, standart konvolüsyonla karşılaştırıldığında:**
```bash
​ParamsDW​​ / Paramsstandard = (Cin ​× 9) / (C_in ​× C_out ​× 9) ​= (1/C_out​​)
```
* Yani C_out kat daha az parametre ve benzer oranda daha düşük hesaplama yükü.

----

## 4. Depthwise Convolution Neyi Çözer?
### 4.1. Uzamsal Filtrelemeyi Dramatik Şekilde Ucuzlatır

* Bir CNN içerisindeki en maliyetli adım olan 3×3 konvolüsyon, depthwise yaklaşımıyla ortalama 8–9 kat daha az parametre ve FLOPs tüketir.

### 4.2. Mobil ve Gömülü Cihazlar İçin Kapı Aralar

* MobileNetV1 (2017)

* MobileNetV2

* MobileNetV3

* EfficientNet (MBConv bloklarının çekirdeği)

* ShuffleNet

gibi tüm hafif mimariler, temel yapıtaşı olarak depthwise convolution kullanmaktadır.


### 4.3. Uzamsal ve Kanal İşlemlerinin Ayrıştırılmasını Sağlar

* Depthwise yalnızca uzamsal işlemi gerçekleştirir.

*Kanal karıştırma işlemi ise pointwise convolution (1×1) ile gerçekleştirilir.*

Bu sayede:

* Mimari tasarım daha modüler hâle gelir,

* Hesaplama maliyeti kontrol edilebilir olur.

---

## 5. Depthwise Convolution’un Sınırlamaları
### 5.1. Kanal Karışımı Yapmaz

* Her kanal yalnızca kendi başına işlenir.
* Dolayısıyla özellikler arasında ilişki kuramaz.

* Bu nedenle depthwise convolution her zaman bir pointwise (1×1) konvolüsyon ile tamamlanır.

### 5.2. Tek Başına Kullanılamaz

* Tek başına bir CNN bloğu değildir.
**Mutlaka bir üst katmanda kanal etkileşimi tanımlanmalıdır.**

---



## 6. Depthwise Convolution’un Araştırma ve Endüstrideki Önemi

* Convolution maliyetini neredeyse bir büyüklük mertebesi azaltmıştır.

* Mobil cihazlarda gerçek zamanlı işlemeyi mümkün kılmıştır.

* Modern lightweight CNN mimarilerinin temel yapı taşı olmuştur.

* Derin öğrenmenin edge computing alanında yaygınlaşmasını sağlamıştır.

**Depthwise convolution olmasaydı:**

* MobilNet modelleri doğamazdı,

* EfficientNet gibi ölçeklenebilir mimariler mümkün olmazdı,

* Edge AI bu kadar hızlı gelişmezdi.


----

## 7. Sonuç

* Depthwise convolution, standart konvolüsyonun maliyet sorununa getirilen yapısal ve oldukça etkili bir çözümdür.
**Mantığı basittir fakat etkisi büyüktür:**

* “Uzamsal filtreleme ucuzdur; kanal karışımı ayrı yapılmalıdır.”

* Bu prensip, günümüz mobil ve verimli CNN mimarilerinin temelini oluşturur.


----

```bash
Input Feature Map (3 channels)
 ┌──────────┐   ┌──────────┐   ┌──────────┐
 │ Channel1 │   │ Channel2 │   │ Channel3 │
 └──────────┘   └──────────┘   └──────────┘
      │               │               │
      ▼               ▼               ▼
   Filter1         Filter2         Filter3
   (3×3)           (3×3)           (3×3)
      │               │               │
      ▼               ▼               ▼
 ┌──────────┐   ┌──────────┐   ┌──────────┐
 │ Out Ch1  │   │ Out Ch2  │   │ Out Ch3  │
 └──────────┘   └──────────┘   └──────────┘


----

---

# Şimdi kodlama işlemine geçelim.

----

## a) Standart 3×3 conv ile karşılaştırma

In [1]:
import torch
import torch.nn as nn

In [2]:
import torch
import torch.nn as nn

# Örnek giriş
batch_size = 1
C_in = 3      # giriş kanal sayısı
C_out = 8     # standart conv için çıkış kanal sayısı
H, W = 32, 32

x = torch.randn(batch_size, C_in, H, W)

# 1) Standart 3x3 konvülasyon
conv_standard = nn.Conv2d(
    in_channels=C_in,
    out_channels=C_out,
    kernel_size=3,
    padding=1,   # boyutu korumak için
    bias=False
)

# 2) Depthwise 3x3 konvülasyon
# groups = in_channels → her kanal ayrı grupla işlenir
conv_depthwise = nn.Conv2d(
    in_channels=C_in,
    out_channels=C_in,   # genelde C_out = C_in
    kernel_size=3,
    padding=1,
    groups=C_in,
    bias=False
)

y_std = conv_standard(x)
y_dw = conv_depthwise(x)

print("Girdi şekli            :", x.shape)
print("Standart Conv çıkışı   :", y_std.shape)
print("Depthwise Conv çıkışı  :", y_dw.shape)

print("Standart Conv parametre sayısı :",
      sum(p.numel() for p in conv_standard.parameters()))
print("Depthwise Conv parametre sayısı:",
      sum(p.numel() for p in conv_depthwise.parameters()))


Girdi şekli            : torch.Size([1, 3, 32, 32])
Standart Conv çıkışı   : torch.Size([1, 8, 32, 32])
Depthwise Conv çıkışı  : torch.Size([1, 3, 32, 32])
Standart Conv parametre sayısı : 216
Depthwise Conv parametre sayısı: 27


Burada:

* Standart conv tüm kanallar arasında tam bağlantı kurar.

* Depthwise conv her kanalı sadece kendi filtresiyle işler.

* Parametre sayısında farkı çıplak gözle görmüş olacaksın.

## b) Depthwise + Pointwise (pratikte kullanılan form)

* Gerçekte modellerde tek başına depthwise değil, arka arkaya depthwise + pointwise kullanılır (depthwise separable conv).

In [3]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, cin, cout, kernel_size=3, stride=1, padding=1):
        super().__init__()
        # 1) Depthwise: uzamsal filtreleme, kanallar ayrık
        self.dw = nn.Conv2d(
            in_channels=cin,
            out_channels=cin,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            groups=cin,
            bias=False
        )
        # 2) Pointwise: 1x1 ile kanal karışımı ve kanal sayısı değişimi
        self.pw = nn.Conv2d(
            in_channels=cin,
            out_channels=cout,
            kernel_size=1,
            bias=False
        )

    def forward(self, x):
        x = self.dw(x)
        x = self.pw(x)
        return x


In [4]:
cin, cout = 32, 64
x = torch.randn(1, cin, 32, 32)

ds_conv = DepthwiseSeparableConv(cin, cout)
y_ds = ds_conv(x)

print("Depthwise Separable çıkışı:", y_ds.shape)
print("Depthwise Separable parametre sayısı:",
      sum(p.numel() for p in ds_conv.parameters()))

Depthwise Separable çıkışı: torch.Size([1, 64, 32, 32])
Depthwise Separable parametre sayısı: 2336


Bu blok:

* MobileNet / EfficientNet gibi modellerde gördüğün yapı.

* “Normal 3×3 conv” yerine çok daha az parametreyle benzer işlev görüyor.

----

### PyTorch tarafında bahsettiğimiz şeyler şunlar:

* C_in → in_channels → Girdi kanal sayısı

* C_out → out_channels → Çıktı kanal sayısı

* groups → Konvolüsyonun kanalları kaça böldüğünü belirleyen parametre

---


## 1) C_in nedir? (in_channels)

* C_in = giriş feature map’indeki kanal sayısı.

Örnekler:

* RGB resim → C_in = 3 (R, G, B)

* Bir CNN’in ara katmanı → C_in = 32, 64, 128 vs. olabilir

PyTorch’ta:
```python
nn.Conv2d(in_channels=C_in, out_channels=..., kernel_size=...)
```


Matematiksel olarak:
```bash
Girdi tensörü: x.shape = (batch_size, C_in, H, W)
```


## 2) C_out nedir? (out_channels)

* C_out = kaç tane çıktı kanalı üretmek istediğin.

Yani:

* Her çıktı kanalı = bir “feature map”

* Her biri, kendi filtre setiyle girdi kanallarını işler.

PyTorch’ta:
```python
nn.Conv2d(in_channels=C_in, out_channels=C_out, kernel_size=3, ...)
```


Çıktı tensörü:
```bash
y.shape = (batch_size, C_out, H_out, W_out)

## 3) groups nedir?

* groups → konvolüsyonda kanalları kaça böldüğünü belirler.

PyTorch’ta Conv2d ağırlık boyutu:

```bash
weight.shape = (C_out ​, C_in​/ groups, kH, kW) 
```
Yani:

**Eğer groups = 1 ise → standart convolution**

* Her çıktı kanalı, tüm giriş kanallarını görür.

**Eğer groups > 1 ise → grouped convolution**

* Giriş kanalları gruplara ayrılır, her grup kendi içinde işlenir.

**Eğer groups = C_in ve C_out = C_in ise → depthwise convolution**

* Her giriş kanalı tek başına bir grup olur, kendi filtresiyle işlenir.

## 4) Örnekle netleştirelim
**Örnek 1 – Standart Conv**
```python
C_in = 3
C_out = 6
conv = nn.Conv2d(C_in, C_out, kernel_size=3, padding=1, groups=1)
```


* Girdi: (N, 3, H, W)

* Çıktı: (N, 6, H, W)

* Her bir çıktı kanalı, 3 giriş kanalının hepsini kullanır.

* Ağırlık boyutu: (6,3,3,3)



## Örnek 2 – Grouped Conv (mesela groups=3)
```python
C_in = 6
C_out = 6
conv = nn.Conv2d(C_in, C_out, kernel_size=3, padding=1, groups=3)
```


* Girdi: (N, 6, H, W)

* groups = 3 → 6 giriş kanalı → 3 gruba bölünür, her grup 2 kanal içerir.

Ağırlık boyutu: (C_out​ , C_in​ / groups ,k ,k)=(6,2,3,3)

Yani:

* grup: in channels [0,1] → out channels [0,1]

* grup: in channels [2,3] → out channels [2,3]

* grup: in channels [4,5] → out channels [4,5]

Her grup kendi içinde çalışır; gruplar birbirine karışmaz.
 

## Örnek 3 – Depthwise Conv (özel bir grouped conv hali)
```python
C_in = 3
C_out = 3
conv_dw = nn.Conv2d(C_in, C_out, kernel_size=3, padding=1, groups=C_in)
```


Burada:

* groups = C_in

Ağırlık boyutu: (3,1,3,3)

* Her giriş kanalının tek başına bir filtresi var.

* Kanal karışımı yok → depthwise convolution dediğimiz şey tam olarak bu.

----
----